# MVD 7. cvičení

## 1. část - Vytvoření dat

V dnešním cvičení je před implementací shlukování potřeba vygenerovat vlastní data. Cílem je vytvořit několik datasetů dle předlohy a vizualizovat je. Postačí dosažení podobného pozice a rozložení bodů.

### 1. dataset - 2 třídy, 100 datových bodů
Použijte funkci ```make_blobs``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html).

![dataset1](expected_outputs/dataset1.png)

Připravený dataset pro další úlohu (k-means) může vypadat například takto:
![prepared](expected_outputs/prepared_dataset.png)

In [259]:
from sklearn.datasets import make_blobs, make_moons, make_circles
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

In [352]:
x1, y1 = make_blobs(n_samples=100, centers=2, n_features=2)
df1 = pd.DataFrame({'x': x1[:,0], 'y': x1[:,1],"label":y1,"kmeans":["TODO"]*100})
fig = px.scatter(df1,x="x",y="y", color=[str(x)  for x in y1])
fig.show()

### 2. dataset - 4 třídy, 200 datových bodů
Použijte funkci ```make_blobs``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html).

![dataset1](expected_outputs/dataset2.png)

In [261]:
x2, y2 = make_blobs(n_samples=200, centers=4, n_features=2)
df2 = pd.DataFrame({'x': x2[:,0], 'y': x2[:,1],"label":y2,"kmeans":["TODO"]*200})
fig = px.scatter(df2,x="x",y="y", color=[str(x)  for x in y2])
fig.show()

### 3. dataset - 2 třídy, 200 datových bodů
Použijte funkci ```make_moons``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html#sklearn.datasets.make_moons).

![dataset1](expected_outputs/dataset3.png)

In [262]:
x3, y3 = make_moons(n_samples=(100,100), noise=0.075)
df3 = pd.DataFrame({'x': x3[:,0], 'y': x3[:,1],"label":y3,"kmeans":["TODO"]*200})
fig = px.scatter(df3,x="x",y="y", color=[str(x)  for x in y3])
fig.show()

### 4. dataset - 2 třídy, 200 datových bodů
Použijte funkci ```make_circles``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html#sklearn.datasets.make_circles).

![dataset1](expected_outputs/dataset4.png)

In [263]:
x4, y4 = make_circles(n_samples=(100,100), noise=0.075,factor=0.2)
df4 = pd.DataFrame({'x': x4[:,0], 'y': x4[:,1],"label":y4,"kmeans":["TODO"]*200})
fig = px.scatter(df4,x="x",y="y", color=[str(x)  for x in y4])
fig.show()

## 2. část - K-means

Implementujte K-means algoritmus, aplikujte ho na všechny 4 datasety a zobrazte výsledky algoritmu. Zároveň se pokuste zvýraznit centroid ve vizualizaci výsledku. Pro aplikaci K-means můžete použít počet tříd v každém datasetu (není třeba hledat parametr K automaticky).

In [274]:
import numpy as np

In [329]:
def kmean(db:pd.DataFrame,k,it = 200) -> tuple:

    p = db[["x","y"]].sample(n=k)
    centroids = np.array([np.array(p["x"]),np.array(p["y"])]).T #init centroids
    data = np.array([np.array(db["x"]),np.array(db["y"])]).T #init data Matrix
    distance = np.zeros((db["x"].shape[0],k)) # init distance
    for i in range(100):
        for j in range(k): # K iteration
            distance[:,j] = np.sum(((centroids[j]-data)**2), axis=1)

        db["kmeans"] = np.argmin(distance, axis=1)
        centroids =  db.groupby(['kmeans']).mean()
        centroids = np.array([np.array(centroids["x"]),np.array(centroids["y"])]).T

    return db, centroids.T



In [330]:
for df_x,k in zip([df1,df2,df3,df4],[2,4,2,2]):
    df_kmean,centroids = kmean(df_x,k)
    fig = px.scatter(df_kmean,x="x",y="y", color=df_kmean["kmeans"].astype(str))
    fig.add_trace(
    go.Scatter(
        x=centroids[0],
        y=centroids[1],
        mode="markers",
        marker=dict(
            color='black',
            size=10),
        showlegend=False
    )

    )

    fig.show()

## Bonus - K-means s nalezenou hodnotou K

Zkuste najít algoritmy, které umožňují hledat hodnotu K pro K-means. Implementujte alespoň jeden z nich a ověřte, že funguje. Ověření proveďte nejen na našich předchozích datasetech, ale zkuste si vygenerovat i dataset s větším množstvím tříd (10-20). (1 bod)

Za implementaci více algoritmů, případně za hlubší analýzu výsledků (např. kdy už přestává hledání hodnoty K správně fungovat) je možné získat ještě jeden bonusový bod.

## Elbow method

- spočítat kvadrát odchylek jednotlivých bodů od svého centroidu
- provést pro 1 až maxK kmeans
- vykreslit do grafu a nalézt tzv. "loket"


In [433]:
def square_error(db:pd.DataFrame, maxK:int) -> list:
    se_list = []
    for k in range(1,maxK+1):
        se = 0 # squere error from centroids
        db,centroids = kmean(db,k)
        centroids = centroids.T
        for index,row in db.iterrows():
            se += np.sum((centroids[int(row["kmeans"])]-np.array([row["x"],row["y"]]))**2)
        se_list.append(se)
    return se_list

def silhouette_and_Fowlkes_Mallows(db:pd.DataFrame, maxK:int) -> tuple[list,list]:
    score_sil = []
    score_fm = []

    for k in range(2,maxK+1):
        db,centroids = kmean(db,k)
        score_fm.append(metrics.fowlkes_mallows_score(db["label"], db["kmeans"]))
        score_sil.append(metrics.silhouette_score(db[["x","y"]], db["kmeans"]))

    return score_sil,score_fm


def plot_graphs(db:pd.DataFrame, square_error_list:list,score_sil:list,score_fm:list):

    fig = px.scatter(db,x="x",y="y", color=[str(x)  for x in y])
    fig.show()

    fig = px.line(
        x = np.arange(1,len(square_error_list)+1),
        y= np.array(se_list),title="Elbow Graph")

    fig.update_layout(
        xaxis_title="počet clusterů",
        yaxis_title="kvadratická odchylka od centroidů",)
    fig.show()
    fig = px.line(
        x = np.arange(2,len(score_sil)+2),
        y= np.array(score_sil),title="Silhouette Coefficient")

    fig.update_layout(
        xaxis_title="počet clusterů",
        yaxis_title="silhouette skóre",)
    fig.show()
    fig = px.line(
        x = np.arange(2,len(score_fm)+2),
        y= np.array(score_fm),title="Fowlkes-Mallows skóre")

    fig.update_layout(
        xaxis_title="počet clusterů",
        yaxis_title="FMI",)
    fig.show()


In [434]:
# init data
x, y = make_blobs(n_samples=100, centers=3, n_features=2)
df = pd.DataFrame({'x': x[:,0], 'y': x[:,1],"label":y,"kmeans":["TODO"]*100})
# plot
se_list = square_error(df1,8)
score_sil,score_fm = silhouette_and_Fowlkes_Mallows(df,8)

plot_graphs(df,se_list,score_sil,score_fm)

In [435]:
x, y = make_moons(n_samples=(100,100), noise=0.075)
df = pd.DataFrame({'x': x[:,0], 'y': x[:,1],"label":y,"kmeans":["TODO"]*200})
se_list = square_error(df,8)
score_sil,score_fm = silhouette_and_Fowlkes_Mallows(df,8)

plot_graphs(df,se_list,score_sil,score_fm)

In [436]:
x, y = make_circles(n_samples=(100,100), noise=0.075,factor=0.2)
df = pd.DataFrame({'x': x[:,0], 'y': x[:,1],"label":y,"kmeans":["TODO"]*200})
# plot
se_list = square_error(df,8)
score_sil,score_fm = silhouette_and_Fowlkes_Mallows(df,8)

plot_graphs(df,se_list,score_sil,score_fm)


In [437]:
# init data
x, y = make_blobs(n_samples=400, centers=13, n_features=2)
df = pd.DataFrame({'x': x[:,0], 'y': x[:,1],"label":y,"kmeans":["TODO"]*400})
# plot
se_list = square_error(df,8)
score_sil,score_fm = silhouette_and_Fowlkes_Mallows(df,8)

plot_graphs(df,se_list,score_sil,score_fm)